# Notebook to the poster: 'Python and Pandas for Papyrology'

this notebook explains in detail how the Network Analysis results displayed on the poster 'Python and Pandas for Papyrology?' were obtained. Keep in mind the code used may not be optimal or pythonic all along, but it is functional. Feel free to use optimize it and to push your own version back

The performed Network Analysis has been 
Czaschez 2016


## Setup

In [1]:
# first, we import the needed modules

import pandas as pd # to visualize and manipulate our data
import csv # to load the data
import networkx as nx # to perform the actual Network Analysis (not visualise it!)
from operator import itemgetter # to get values from single nodes 
import ast # to transform the lemma list in a string
import seaborn as sns #to colorize our final df


Bad key "text.kerning_factor" on line 4 in
C:\Users\Audric\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


## Functions

In [2]:
def do_all(a_str):
    a_list = a_str.split(' ')
    new_list = []
    for p in a_list:
        new_list.append(p)
    a_df = pd.DataFrame(new_list).transpose()
    a_df = a_df.rename(columns ={0:'lemma'})
    
    return a_df

def make_df2(a_df):
    source_list = []
    target_list = []
    for index, v in a_df.lemma.items():
        try:
            target_list.append(a_df.lemma[index+1])
            source_list.append(v)
        except:
            continue
    
        try:
            target_list.append(a_df.lemma[index-1])
            source_list.append(v)
        except:
            continue

    target_list.append(a_df.lemma.iloc[-2])
    source_list.append(a_df.lemma.iloc[-1])
    edges = [source_list, target_list]
    df2 = pd.DataFrame(edges).transpose()
    df2 = df2.rename(columns ={0:'source', 1:'target'})
    return df2

def listToString(s):  
    
    str1 = " "   
    return (str1.join(s))

def make_string(x):
    x = ast.literal_eval(x)
    x = (listToString(x))
    return x

# this last functions performs the actual NA

def from_string_toNADF(a_str):
    testdf = do_all(a_str)
    testdf2 = make_df2(testdf)

    testdf.to_csv('nodesdepassage.csv', index=False)
    testdf2.to_csv('edgesdepassage.csv', index=False)

#from csv to graph part
    with open('nodesdepassage.csv', encoding='utf-8') as nodecsv:
        nodereader = csv.reader(nodecsv) # Read the csv
    # Retrieve the data (using Python list comprhension and list slicing to remove the header row, see footnote 3)
        nodes = [n for n in nodereader][1:]

        node_names = [n[0] for n in nodes] # Get a list of only the node names

    with open('edgesdepassage.csv', encoding='utf-8') as edgecsv: # Open the file
        edgereader = csv.reader(edgecsv) # Read the csv
        edges = [tuple(e) for e in edgereader][1:] # Retrieve the data
    
    a = len(edges)
    b = len(node_names)

# NA part
    G = nx.Graph()
    G.add_nodes_from(node_names)
    G.add_edges_from(edges)
    infos = nx.info(G)
    degree_dict = dict(G.degree(G.nodes()))
    nx.set_node_attributes(G, degree_dict, 'degree')
    sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)
    c = sorted_degree[0]
    betweenness_dict = nx.betweenness_centrality(G) # Run betweenness centrality
    #eigenvector_dict = nx.eigenvector_centrality(G) # Run eigenvector centrality

    # Assign each to an attribute in your network
    nx.set_node_attributes(G, betweenness_dict, 'betweenness')
    #nx.set_node_attributes(G, eigenvector_dict, 'eigenvector')
    sorted_betweenness = sorted(betweenness_dict.items(), key=itemgetter(1), reverse=True)
    d = sorted_betweenness[0]

    #sorted_eigen = sorted(eigenvector_dict.items(), key=itemgetter(1), reverse=True)
    #e = sorted_eigen[0]
    G.clear()
    return d[1]

## Load the data into pandas

In [3]:
# We will load a csv table 

df1 = pd.read_csv('dflem.csv')
df1['OBJECT']= df1.LEMMAS.apply(lambda x: make_string(x))

In [4]:
# you can already have a deeper look at the data if you like to
df1

,Unnamed: 0,Unnamed: 0.1,NAME,DATA,METADATA,PLACE,CENTURY,TOKENS,LEMMAS,OBJECT
0,0,0,/apf;63;138,ταρία τῷ ἀδελφῷ μου ἀπολλῷ χαίρειν ἐβάσταζέ μ...,"[<table class=""metadata"">\n<tbody>\n<tr>\n<th ...",arsi,2,"['ταρία', 'τῷ', 'ἀδελφῷ', 'μου', 'ἀπολλῷ', 'χα...","['ταρία', 'ἀδελφός', 'ἐγώ', 'ἀπόλάω1', 'χαίρω'...",ταρία ἀδελφός ἐγώ ἀπόλάω1 χαίρω ἐβάσταζέ ἐγώ π...
1,1,1,/basp;49;83,αὐρήλειος πωλείον στρατειώτης λεγειῶνος β βοη...,"[<table class=""metadata"">\n<tbody>\n<tr>\n<th ...",other,3,"['αὐρήλειος', 'πωλείον', 'στρατειώτης', 'λεγει...","['αὐρήλειος', 'πωλείον', 'στρατειώτης', 'λεγει...",αὐρήλειος πωλείον στρατειώτης λεγειῶνος β βοηθ...
2,2,2,/basp;49;135,πανίσκος ἀϊῶνι τῷ ἀδελφῶι πολλὰ χαίρειν πρὸ μ...,"[<table class=""metadata"">\n<tbody>\n<tr>\n<th ...",other,3,"['πανίσκος', 'ἀϊῶνι', 'τῷ', 'ἀδελφῶι', 'πολλὰ'...","['πανίσκος', 'ἀϊῶνι', 'ἀδελφός', 'πολλὰ', 'χαί...",πανίσκος ἀϊῶνι ἀδελφός πολλὰ χαίρω πρὸ πᾶς εὔ...
3,3,3,/bgu;1;261,θερμουθᾶς ἀπολιναρίῳ τῷ ἀδελφῷ πλεῖστα χαίρει...,"[<table class=""metadata"">\n<tbody>\n<tr>\n<th ...",arsi,0,"['θερμουθᾶς', 'ἀπολιναρίῳ', 'τῷ', 'ἀδελφῷ', 'π...","['θερμουθᾶς', 'ἀπολιναρίῳ', 'ἀδελφός', 'πλεῖσ...",θερμουθᾶς ἀπολιναρίῳ ἀδελφός πλεῖστος χαίρω γ...
4,4,4,/bgu;3;822,θερμουτᾶς ἀπολιναρίῳ τῶι ἀδελφῷ χαίρειν γινωσ...,"[<table class=""metadata"">\n<tbody>\n<tr>\n<th ...",arsi,0,"['θερμουτᾶς', 'ἀπολιναρίῳ', 'τῶι', 'ἀδελφῷ', '...","['θερμουτᾶς', 'ἀπολιναρίῳ', 'ἀδελφός', 'χαίρω'...",θερμουτᾶς ἀπολιναρίῳ ἀδελφός χαίρω γινωσκιν σύ...
...,...,...,...,...,...,...,...,...,...,...
130,130,130,/sb;16;12980,διὰ τοῦ παρόντος ἡμετέρου γράμματος πλεῖστα ὡ...,"[<table class=""metadata"">\n<tbody>\n<tr>\n<th ...",other,6,"['διὰ', 'τοῦ', 'παρόντος', 'ἡμετέρου', 'γράμμα...","['πάρειμι1', 'ἡμέτερος', 'γράμμα', 'πλεῖστος'...",πάρειμι1 ἡμέτερος γράμμα πλεῖστος παρὼν προσκ...
131,131,131,/sb;16;12981,ἶσις σεραπίωνι τῶι ἀδελφῶι πολλὰ χαίρειν τὸ π...,"[<table class=""metadata"">\n<tbody>\n<tr>\n<th ...",other,2,"['ἶσις', 'σεραπίωνι', 'τῶι', 'ἀδελφῶι', 'πολλὰ...","['ἶσις', 'σεραπίωνι', 'ἀδελφός', 'πολλὰ', 'χαί...",ἶσις σεραπίωνι ἀδελφός πολλὰ χαίρω προσκύνημά ...
132,132,132,/sb;20;14132,πτολέμα βελήουτι τῇ μητρεὶ καὶ κυρί ᾳ πλεῖστα...,"[<table class=""metadata"">\n<tbody>\n<tr>\n<th ...",other,1,"['πτολέμα', 'βελήουτι', 'τῇ', 'μητρεὶ', 'καὶ',...","['πτολέμα', 'βελήουτι', 'μητρεὶ', 'κυρί', 'ᾳ',...",πτολέμα βελήουτι μητρεὶ κυρί ᾳ πλεῖστος χαίρω...
133,133,133,/sb;22;15768,ὑπατείας τῶν δεσποτῶν ἡμῶν ἰοουινιανοῦ αἰωνίο...,"[<table class=""metadata"">\n<tbody>\n<tr>\n<th ...",other,4,"['ὑπατείας', 'τῶν', 'δεσποτῶν', 'ἡμῶν', 'ἰοουι...","['ὑπατεία', 'δεσπότης', 'ἐγώ', 'ἰοουινιανοῦ', ...",ὑπατεία δεσπότης ἐγώ ἰοουινιανοῦ αἰώνιος αὔγ...


In [5]:
# we will remove the greek stopwords according to the .txt file of Classical Language Toolkit (cltk)
f = open("stopwords_greek.txt", encoding='utf-8')
cltk_stopwords = f.read()
cltk_stopwords = cltk_stopwords.replace('\n', ' ')

def stop_them(a_str):
    a_list = a_str.split(' ')
    for e in a_list:
        if e in cltk_stopwords:
            a_list.remove(e)
    
    a_str = listToString(a_list)
    return a_str

df1['OBJECT']= df1.OBJECT.apply(lambda x: stop_them(x))

In [6]:
dfa = df1[df1.CENTURY != -3]
dfb = dfa[df1.CENTURY != -2]
dfc = dfb[df1.CENTURY != 4]
dfd = dfc[df1.CENTURY != 5]
dfe = dfd[df1.CENTURY != 6]
dff = dfe[df1.CENTURY != 7]
dfg = dff[df1.CENTURY != 8]
dfright_time = dfg
dt = dfright_time

C:\Users\Audric\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\Audric\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Audric\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\Users\Audric\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\Audric\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\Audric\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame ind

In [7]:
del dt['DATA']
del dt['Unnamed: 0']
del dt['Unnamed: 0.1']
del dt['METADATA']
del dt['TOKENS']
del dt['LEMMAS']
dt = dt.reset_index()
del dt['index']


In [9]:
# works fine up to here

In [8]:
dt['betweenness#'] = dt.OBJECT.apply(from_string_toNADF)
dt[dt.top1betweenness == dt.top1degree]
dt.to_csv('ouf.csv')
dshow = dt
dshow = dt
del dshow['OBJECT']
dshow
dshow.NAME = dshow.NAME.apply(lambda x: cutone(x))
dshow
dshow.groupby(by='PLACE').mean()

IndexError: single positional indexer is out-of-bounds

In [ ]:
cm = sns.light_palette("red", as_cmap=True)

s = dshow.sort_values(by='degree#', ascending=False).head(20).style.background_gradient(cmap=cm)
s

In [ ]:
dshow.sort_values(by='degree#', ascending=False).head(20)
dshow.groupby(by='CENTURY').mean()

In [ ]:
def cutone(a_str):
    return a_str[1:]

In [ ]:
dshow.answers = dshow.answers.astype(str)

In [ ]:
dshow.answers